In [ ]:
testcase = """
1) Cho tôi GMV theo từng tháng trong toàn bộ dữ liệu, kèm tỷ lệ tăng trưởng MoM và YoY cho mỗi tháng; sắp xếp theo thời gian để thấy xu hướng.
WITH bounds AS (
  SELECT date_trunc('month', MAX(order_purchase_timestamp)) + interval '1 month' AS max_month_next
  FROM olist.olist_orders
),
win AS (
  SELECT (max_month_next - interval '6 months') AS recent_start,
         max_month_next                         AS recent_end,
         (max_month_next - interval '12 months') AS prev_start,
         (max_month_next - interval '6 months')  AS prev_end
  FROM bounds
),
order_price AS (
  SELECT order_id, SUM(price) AS order_total_price
  FROM olist.olist_order_items
  GROUP BY 1
),
order_pay AS (
  SELECT order_id, SUM(payment_value) AS order_pay_value
  FROM olist.olist_order_payments
  GROUP BY 1
),
alloc AS (  -- phân bổ payment theo tỷ trọng giá item
  SELECT o.order_id, p.product_category_name,
         (oi.price / NULLIF(op.order_total_price,0)) * pay.order_pay_value AS alloc_value,
         o.order_purchase_timestamp
  FROM olist.olist_orders o
  JOIN olist.olist_order_items oi USING(order_id)
  JOIN olist.olist_products p USING(product_id)
  JOIN order_price op USING(order_id)
  JOIN order_pay   pay USING(order_id)
  WHERE o.order_status = 'delivered'
),
recent AS (
  SELECT a.product_category_name, SUM(a.alloc_value) AS gmv_recent
  FROM alloc a, win w
  WHERE a.order_purchase_timestamp >= w.recent_start
    AND a.order_purchase_timestamp <  w.recent_end
  GROUP BY 1
),
prev AS (
  SELECT a.product_category_name, SUM(a.alloc_value) AS gmv_prev
  FROM alloc a, win w
  WHERE a.order_purchase_timestamp >= w.prev_start
    AND a.order_purchase_timestamp <  w.prev_end
  GROUP BY 1
)
SELECT COALESCE(r.product_category_name,'UNKNOWN') AS product_category_name,
       r.gmv_recent,
       p.gmv_prev,
       (r.gmv_recent - COALESCE(p.gmv_prev,0)) AS gmv_delta,
       (r.gmv_recent / NULLIF(p.gmv_prev,0) - 1) AS growth_rate
FROM recent r
LEFT JOIN prev p USING (product_category_name)
ORDER BY gmv_delta DESC NULLS LAST;

2) Phân tích giữ chân khách hàng theo cohort tháng đầu mua (cohort_month): với các đơn “delivered”, tính retention rate theo month_index (0,1,2,…) và quy mô cohort.
WITH bounds AS (
  SELECT date_trunc('month', MAX(order_purchase_timestamp)) + interval '1 month' AS max_month_next
  FROM olist.olist_orders
),
win AS (
  SELECT (max_month_next - interval '6 months') AS recent_start,
         max_month_next                         AS recent_end,
         (max_month_next - interval '12 months') AS prev_start,
         (max_month_next - interval '6 months')  AS prev_end
  FROM bounds
),
-- Lặp lại phần xếp hạng category để bắt được ngành dẫn đầu
order_price AS (
  SELECT order_id, SUM(price) AS order_total_price
  FROM olist.olist_order_items GROUP BY 1
),
order_pay AS (
  SELECT order_id, SUM(payment_value) AS order_pay_value
  FROM olist.olist_order_payments GROUP BY 1
),
alloc AS (
  SELECT o.order_id, p.product_id, p.product_category_name,
         (oi.price / NULLIF(op.order_total_price,0)) * pay.order_pay_value AS alloc_value,
         o.order_purchase_timestamp, o.order_delivered_customer_date, o.order_estimated_delivery_date
  FROM olist.olist_orders o
  JOIN olist.olist_order_items oi USING(order_id)
  JOIN olist.olist_products p USING(product_id)
  JOIN order_price op USING(order_id)
  JOIN order_pay   pay USING(order_id)
  WHERE o.order_status = 'delivered'
),
cat_recent AS (
  SELECT product_category_name, SUM(alloc_value) AS gmv_recent
  FROM alloc, win
  WHERE order_purchase_timestamp >= win.recent_start
    AND order_purchase_timestamp <  win.recent_end
  GROUP BY 1
),
cat_prev AS (
  SELECT product_category_name, SUM(alloc_value) AS gmv_prev
  FROM alloc, win
  WHERE order_purchase_timestamp >= win.prev_start
    AND order_purchase_timestamp <  win.prev_end
  GROUP BY 1
),
top_cat AS (
  SELECT cr.product_category_name
  FROM cat_recent cr
  LEFT JOIN cat_prev cp USING(product_category_name)
  ORDER BY (cr.gmv_recent - COALESCE(cp.gmv_prev,0)) DESC NULLS LAST
  LIMIT 1
),
per_order_inst AS (  -- installments theo đơn
  SELECT order_id, COALESCE(MAX(payment_installments),1) AS installments
  FROM olist.olist_order_payments
  GROUP BY 1
),
recent_orders AS (   -- chỉ đơn 6M gần nhất, thuộc ngành top
  SELECT a.*
  FROM alloc a, win w, top_cat tc
  WHERE a.product_category_name = tc.product_category_name
    AND a.order_purchase_timestamp >= w.recent_start
    AND a.order_purchase_timestamp <  w.recent_end
),
per_product AS (
  SELECT ro.product_id,
         SUM(ro.alloc_value) AS gmv,
         COUNT(DISTINCT ro.order_id) AS orders,
         AVG( (pi.installments > 1)::numeric ) AS share_installments_gt1,
         AVG( (ro.order_delivered_customer_date > ro.order_estimated_delivery_date)::numeric ) AS late_rate
  FROM recent_orders ro
  LEFT JOIN per_order_inst pi USING(order_id)
  GROUP BY ro.product_id
),
reviews AS (
  SELECT oi.product_id,
         percentile_cont(0.5) WITHIN GROUP (ORDER BY r.review_score) AS median_review
  FROM olist.olist_order_reviews r
  JOIN olist.olist_order_items oi USING(order_id)
  JOIN recent_orders ro ON ro.order_id = oi.order_id AND ro.product_id = oi.product_id
  GROUP BY oi.product_id
)
SELECT p.product_id,
       pr.product_category_name,
       COALESCE(t.product_category_name_english, INITCAP(REPLACE(pr.product_category_name,'_',' '))) AS category_en,
       per.gmv,
       per.orders,
       per.late_rate,
       COALESCE(rev.median_review, NULL) AS median_review,
       per.share_installments_gt1
FROM per_product per
JOIN olist.olist_products pr USING(product_id)
LEFT JOIN reviews rev USING(product_id)
LEFT JOIN olist.olist_product_category_translation t
       ON t.product_category_name = pr.product_category_name
ORDER BY per.gmv DESC
LIMIT 15;

3) Hãy phân khúc khách hàng theo RFM (Recency/Frequency/Monetary) bằng cách chấm điểm theo quintile (1–5) cho từng trục; gán nhãn segment (Champions, Loyal, At Risk, Hibernating, Others) và báo cáo số khách + tổng chi tiêu theo segment.
WITH bounds AS (
  SELECT date_trunc('month', MAX(order_purchase_timestamp)) + interval '1 month' AS max_month_next
  FROM olist.olist_orders
),
win AS (
  SELECT (max_month_next - interval '6 months') AS recent_start,
         max_month_next                         AS recent_end,
         (max_month_next - interval '12 months') AS prev_start,
         (max_month_next - interval '6 months')  AS prev_end
  FROM bounds
),
order_price AS (
  SELECT order_id, SUM(price) AS order_total_price
  FROM olist.olist_order_items GROUP BY 1
),
order_pay AS (
  SELECT order_id, SUM(payment_value) AS order_pay_value
  FROM olist.olist_order_payments GROUP BY 1
),
alloc AS (
  SELECT o.order_id, oi.seller_id, p.product_category_name,
         (oi.price / NULLIF(op.order_total_price,0)) * pay.order_pay_value AS alloc_value,
         o.order_purchase_timestamp
  FROM olist.olist_orders o
  JOIN olist.olist_order_items oi USING(order_id)
  JOIN olist.olist_products p USING(product_id)
  JOIN order_price op USING(order_id)
  JOIN order_pay   pay USING(order_id)
  WHERE o.order_status = 'delivered'
),
cat_recent AS (
  SELECT product_category_name, SUM(alloc_value) AS gmv_recent
  FROM alloc, win
  WHERE order_purchase_timestamp >= win.recent_start
    AND order_purchase_timestamp <  win.recent_end
  GROUP BY 1
),
cat_prev AS (
  SELECT product_category_name, SUM(alloc_value) AS gmv_prev
  FROM alloc, win
  WHERE order_purchase_timestamp >= win.prev_start
    AND order_purchase_timestamp <  win.prev_end
  GROUP BY 1
),
top_cat AS (
  SELECT cr.product_category_name
  FROM cat_recent cr
  LEFT JOIN cat_prev cp USING(product_category_name)
  ORDER BY (cr.gmv_recent - COALESCE(cp.gmv_prev,0)) DESC NULLS LAST
  LIMIT 1
),
seller_recent AS (
  SELECT a.seller_id, SUM(a.alloc_value) AS gmv_recent
  FROM alloc a, win w, top_cat tc
  WHERE a.product_category_name = tc.product_category_name
    AND a.order_purchase_timestamp >= w.recent_start
    AND a.order_purchase_timestamp <  w.recent_end
  GROUP BY 1
),
seller_prev AS (
  SELECT a.seller_id, SUM(a.alloc_value) AS gmv_prev
  FROM alloc a, win w, top_cat tc
  WHERE a.product_category_name = tc.product_category_name
    AND a.order_purchase_timestamp >= w.prev_start
    AND a.order_purchase_timestamp <  w.prev_end
  GROUP BY 1
),
delta AS (
  SELECT COALESCE(sr.seller_id, sp.seller_id) AS seller_id,
         COALESCE(sr.gmv_recent,0) AS gmv_recent,
         COALESCE(sp.gmv_prev,0)   AS gmv_prev,
         COALESCE(sr.gmv_recent,0) - COALESCE(sp.gmv_prev,0) AS gmv_delta
  FROM seller_recent sr
  FULL JOIN seller_prev sp USING(seller_id)
),
pos AS (  -- chỉ giữ đóng góp dương
  SELECT * FROM delta WHERE gmv_delta > 0
),
tot AS (SELECT SUM(gmv_delta) AS total_delta FROM pos),
ranked AS (
  SELECT p.*,
         SUM(p.gmv_delta) OVER (ORDER BY p.gmv_delta DESC) / NULLIF(t.total_delta,0) AS cum_share,
         ROW_NUMBER() OVER (ORDER BY p.gmv_delta DESC) AS rn
  FROM pos p CROSS JOIN tot t
)
SELECT seller_id, gmv_recent, gmv_prev, gmv_delta, cum_share
FROM ranked
WHERE rn <= 10 OR cum_share <= 0.60
ORDER BY gmv_delta DESC;

4) Điểm review ảnh hưởng thế nào tới khả năng mua lại trong 90 ngày? Với các đơn “delivered”, tính tỷ lệ mua lại ≤90 ngày tiếp theo theo từng mức review_score (1–5).
-- SQL 4) Ảnh hưởng review_score tới tỷ lệ mua lại trong 90 ngày
WITH o AS (
  SELECT o.order_id, o.customer_id, o.order_purchase_timestamp,
         LEAD(o.order_purchase_timestamp) OVER (PARTITION BY o.customer_id ORDER BY o.order_purchase_timestamp) AS next_purchase_ts
  FROM olist.olist_orders o
  WHERE o.order_status = 'delivered'
),
base AS (
  SELECT o.order_id, r.review_score,
         (CASE WHEN o.next_purchase_ts IS NOT NULL
                     AND o.next_purchase_ts <= o.order_purchase_timestamp + INTERVAL '90 days'
               THEN 1 ELSE 0 END) AS repurchase_90d
  FROM o
  LEFT JOIN olist.olist_order_reviews r USING(order_id)
)
SELECT review_score,
       COUNT(*) AS orders,
       AVG(repurchase_90d::numeric) AS repurchase_rate_90d
FROM base
GROUP BY 1
ORDER BY 1;

5) Khoảng cách vận chuyển (Haversine giữa zip seller ↔ customer) ảnh hưởng đến lead time ra sao? Bucket theo decile khoảng cách và trả về số đơn, p50/p90 lead time (ngày), khoảng cách trung bình cho từng decile.
-- SQL 5) Khoảng cách vận chuyển (Haversine) vs lead time (p50/p90) theo decile
WITH geo AS (
  SELECT geolocation_zip_code_prefix AS zip,
         AVG(geolocation_lat)  AS lat,
         AVG(geolocation_lng)  AS lng
  FROM olist.olist_geolocation
  GROUP BY 1
),
pairs AS (
  SELECT oi.order_id, oi.seller_id, o.customer_id,
         cs.customer_zip_code_prefix::text AS cz,
         ss.seller_zip_code_prefix::text   AS sz,
         o.order_purchase_timestamp, o.order_delivered_customer_date,
         GREATEST(1, (o.order_delivered_customer_date::date - o.order_purchase_timestamp::date))::int AS days
  FROM olist.olist_order_items oi
  JOIN olist.olist_orders o      USING(order_id)
  JOIN olist.olist_customers cs  USING(customer_id)
  JOIN olist.olist_sellers  ss   USING(seller_id)
  WHERE o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
),
dist AS (
  SELECT p.*,
         gcz.lat AS c_lat, gcz.lng AS c_lng,
         gsz.lat AS s_lat,  gsz.lng AS s_lng,
         2*6371*ASIN(
           SQRT( POWER(SIN(radians(gcz.lat - gsz.lat)/2),2)
               + COS(radians(gsz.lat))*COS(radians(gcz.lat))
               * POWER(SIN(radians(gcz.lng - gsz.lng)/2),2) )
         ) AS km
  FROM pairs p
  JOIN geo gcz ON gcz.zip = p.cz
  JOIN geo gsz ON gsz.zip = p.sz
),
bucketed AS (
  SELECT *,
         NTILE(10) OVER (ORDER BY km) AS distance_decile
  FROM dist
)
SELECT distance_decile,
       COUNT(*) AS orders,
       percentile_cont(0.5) WITHIN GROUP (ORDER BY days) AS p50_days,
       percentile_cont(0.9) WITHIN GROUP (ORDER BY days) AS p90_days,
       AVG(km) AS avg_km
FROM bucketed
GROUP BY 1
ORDER BY 1;

6) Trả góp (installments) liên quan thế nào tới AOV và chất lượng dịch vụ? Tính AOV trung bình và điểm review trung bình theo số kỳ trả góp; đồng thời tính hệ số tương quan giữa số kỳ trả góp và AOV.
-- SQL 6a) Trả góp (installments) ↔ AOV & review trung bình
WITH per_order AS (
  SELECT o.order_id,
         COALESCE(MAX(op.payment_installments),1) AS installments,
         SUM(op.payment_value) AS order_value
  FROM olist.olist_orders o
  JOIN olist.olist_order_payments op USING(order_id)
  WHERE o.order_status IN ('delivered','shipped','invoiced','processing','created','approved')
  GROUP BY 1
)
SELECT p.installments,
       COUNT(*) AS orders,
       AVG(p.order_value) AS avg_aov,
       AVG(r.review_score::numeric) AS avg_review
FROM per_order p
LEFT JOIN olist.olist_order_reviews r ON r.order_id = p.order_id
GROUP BY 1
ORDER BY installments;

-- SQL 6b) Hệ số tương quan installments ~ AOV
SELECT regr_sxy(order_value, installments)
       / NULLIF(SQRT(regr_sxx(order_value, installments)*regr_syy(order_value, installments)),0) AS corr_installments_aov
FROM (
  SELECT COALESCE(MAX(op.payment_installments),1) AS installments,
         SUM(op.payment_value) AS order_value
  FROM olist.olist_order_payments op
  JOIN olist.olist_orders o USING(order_id)
  GROUP BY o.order_id
) t;

7) Mức độ tập trung doanh thu: nhóm top 1% seller chiếm bao nhiêu % GMV toàn sàn (phân bổ doanh thu theo tỷ trọng item trên mỗi đơn)?
-- SQL 7) Top 1% seller chiếm bao nhiêu % GMV (phân bổ theo tỷ trọng item)
WITH order_price AS (
  SELECT order_id, SUM(price) AS order_total_price
  FROM olist.olist_order_items
  GROUP BY 1
),
seller_rev AS (
  SELECT oi.seller_id,
         SUM( (oi.price / NULLIF(opr.order_total_price,0)) * op.payment_value ) AS seller_gmv
  FROM olist.olist_order_items oi
  JOIN order_price opr USING(order_id)
  JOIN olist.olist_order_payments op USING(order_id)
  GROUP BY 1
),
ranked AS (
  SELECT s.*,
         NTILE(100) OVER (ORDER BY seller_gmv DESC) AS pct_rank
  FROM seller_rev s
),
tot AS (SELECT SUM(seller_gmv) AS total_gmv FROM seller_rev)
SELECT
  SUM(CASE WHEN pct_rank=1 THEN seller_gmv ELSE 0 END) / t.total_gmv AS share_top_1pct
FROM ranked r CROSS JOIN tot t;

8) “Biên gộp” theo ngành hàng (proxy = sum(price) − sum(freight_value)): liệt kê revenue, freight, margin và margin% cho từng category; xếp hạng top 20 category margin% cao nhất (ưu tiên tên tiếng Anh nếu có bản dịch).
-- SQL 8) Biên gộp (proxy = sum(price) - sum(freight)) & margin% theo category
WITH cat AS (
  SELECT p.product_category_name,
         SUM(oi.price)          AS revenue,
         SUM(oi.freight_value)  AS freight,
         SUM(oi.price - oi.freight_value) AS margin
  FROM olist.olist_order_items oi
  JOIN olist.olist_products p USING(product_id)
  GROUP BY 1
)
SELECT COALESCE(t.product_category_name_english, INITCAP(REPLACE(c.product_category_name,'_',' '))) AS category,
       c.revenue, c.freight, c.margin,
       (c.margin / NULLIF(c.revenue,0)) AS margin_pct
FROM cat c
LEFT JOIN olist.olist_product_category_translation t
       ON t.product_category_name = c.product_category_name
ORDER BY margin_pct DESC NULLS LAST
LIMIT 20;

9) Chất lượng vận hành theo quy mô seller: chia seller thành 10 decile theo GMV và báo cáo tỉ lệ hủy đơn (canceled rate) và tỉ lệ giao trễ (delivered muộn so với estimated) của mỗi decile.
-- SQL 9) Chất lượng vận hành theo decile GMV của seller: cancel_rate & late_rate
WITH seller_gmv AS (
  SELECT oi.seller_id, SUM(oi.price) AS gmv
  FROM olist.olist_order_items oi
  GROUP BY 1
),
bucket AS (
  SELECT seller_id, gmv,
         NTILE(10) OVER (ORDER BY gmv DESC) AS gmv_decile
  FROM seller_gmv
),
kpi AS (
  SELECT b.gmv_decile,
         COUNT(DISTINCT o.order_id) FILTER (WHERE o.order_status='canceled') AS canceled_orders,
         COUNT(DISTINCT o.order_id) FILTER (WHERE o.order_status='delivered') AS delivered_orders,
         COUNT(*) FILTER (WHERE o.order_status='delivered'
                          AND o.order_delivered_customer_date > o.order_estimated_delivery_date) AS late_deliveries
  FROM bucket b
  JOIN olist.olist_order_items oi ON oi.seller_id = b.seller_id
  JOIN olist.olist_orders o        ON o.order_id  = oi.order_id
  GROUP BY 1
)
SELECT gmv_decile,
       canceled_orders::numeric / NULLIF(delivered_orders + canceled_orders,0) AS cancel_rate,
       late_deliveries::numeric / NULLIF(delivered_orders,0)                   AS late_rate
FROM kpi
ORDER BY gmv_decile;

10) Ảnh hưởng “giỏ hàng rộng”: nhóm đơn theo số lượng ngành hàng khác nhau trong một đơn (distinct categories) và trả về số đơn, AOV trung bình, p50 và p90 giá trị đơn cho từng mức “độ rộng giỏ”.
-- SQL 10) “Giỏ hàng rộng” (distinct categories trong đơn) ↔ giá trị đơn
WITH order_cat AS (
  SELECT oi.order_id,
         COUNT(DISTINCT p.product_category_name) AS distinct_cats,
         SUM(oi.price) AS order_items_value
  FROM olist.olist_order_items oi
  JOIN olist.olist_products p USING(product_id)
  GROUP BY 1
),
with_pay AS (
  SELECT oc.order_id, oc.distinct_cats,
         SUM(op.payment_value) AS order_value
  FROM order_cat oc
  JOIN olist.olist_order_payments op USING(order_id)
  GROUP BY 1,2
)
SELECT distinct_cats,
       COUNT(*) AS orders,
       AVG(order_value) AS avg_order_value,
       percentile_cont(0.5) WITHIN GROUP (ORDER BY order_value) AS p50_order_value,
       percentile_cont(0.9) WITHIN GROUP (ORDER BY order_value) AS p90_order_value
FROM with_pay
GROUP BY 1
ORDER BY 1;

"""